In [ ]:
pip install tfx

In [4]:
import os
import tensorflow as tf

from tfx.components import CsvExampleGen
from tfx.components import ExampleValidator
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Transform
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from google.protobuf.json_format import MessageToDict

import pprint

pp = pprint.PrettyPrinter()

In [14]:
from google.colab import drive
drive.mount('./content')

Drive already mounted at ./content; to attempt to forcibly remount, call drive.mount("./content", force_remount=True).


In [15]:
_pipeline_root = 'metro_traffic_pipeline/'
_data_root = '/content/content/MyDrive/mlops/course2/week2/data2'
_data_filepath = os.path.join(_data_root, 'Metro_Interstate_Traffic_Volume.csv')


In [16]:
!head {_data_filepath}

holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume
None,288.28,0.0,0.0,40,Clouds,scattered clouds,2012-10-02 09:00:00,5545
None,289.36,0.0,0.0,75,Clouds,broken clouds,2012-10-02 10:00:00,4516
None,289.58,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 11:00:00,4767
None,290.13,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 12:00:00,5026
None,291.14,0.0,0.0,75,Clouds,broken clouds,2012-10-02 13:00:00,4918
None,291.72,0.0,0.0,1,Clear,sky is clear,2012-10-02 14:00:00,5181
None,293.17,0.0,0.0,1,Clear,sky is clear,2012-10-02 15:00:00,5584
None,293.86,0.0,0.0,1,Clear,sky is clear,2012-10-02 16:00:00,6015
None,294.14,0.0,0.0,20,Clouds,few clouds,2012-10-02 17:00:00,5791


In [17]:
context = InteractiveContext(pipeline_root=_pipeline_root)

In [18]:
example_gen = CsvExampleGen(input_base=_data_root)

In [19]:
context.run(example_gen)


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 2
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))